In [ ]:
!ls

cache_dir  outputs  sample_data   train_iob.txt
drive	   runs     test_iob.txt  valid_iob.txt


In [ ]:
!cp -r /content/outputs /content/drive/MyDrive/IOB_MODEL

In [ ]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, t) for w, t in zip(s['words'].values.tolist(),
                                                       s['labels'].values.tolist())]
        self.grouped = self.df.groupby("sentence_id").apply(agg)
        self.sentences = [s for s in self.grouped]

    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

## Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import xlrd

In [ ]:
# data = pd.read_excel('/content/drive/MyDrive/output.xlsx')


data = pd.read_excel('/content/drive/MyDrive/GFG/output.xlsx')

In [ ]:
data.head(10)

,Unnamed: 0,CHECK,0,1,2,3,4,5,6,7,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,0,NaN,प्रतिनिधी,नागपूरपानठेले,बंद,असतानाही,सध्या,मोठ्या,प्रमाणात,खर्रा,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,ED,NEO,O,O,O,O,O,O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NaN,बंदी,असताना,गुटखाही,नागपुरात,मोठ्या,प्रमाणात,येत,असल्याचेही,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,O,O,O,NEL,O,O,O,O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,1,अन्न,व,औषध,प्रशासन,विभागाने,केलेल्या,तपासणीत,७१,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,O,O,O,BNEO,INEO,O,O,NEM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3,NaN,सहआयुक्त,चंद्रकांत,पवार,यांच्या,मार्गदर्शनाखाली,या,कारवाया,करण्यात,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75003 entries, 0 to 75002
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   25002 non-null  object 
 1   CHECK        4544 non-null   object 
 2   0            49967 non-null  object 
 3   1            49984 non-null  object 
 4   2            49983 non-null  object 
 5   3            49980 non-null  object 
 6   4            49960 non-null  object 
 7   5            49665 non-null  object 
 8   6            43807 non-null  object 
 9   7            36999 non-null  object 
 10  8            29848 non-null  object 
 11  9            22867 non-null  object 
 12  10           16230 non-null  object 
 13  11           10169 non-null  object 
 14  12           4647 non-null   object 
 15  Unnamed: 15  101 non-null    object 
 16  Unnamed: 16  8 non-null      object 
 17  Unnamed: 17  4 non-null      object 
 18  Unnamed: 18  2 non-null      object 
 19  Unna

In [ ]:
drop_columns=['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25']

In [ ]:
data_1= data.drop(drop_columns, axis=1)

In [ ]:
data_1.head(20)

,Unnamed: 0,CHECK,0,1,2,3,4,5,6,7,8,9,10,11,12,Unnamed: 15
0,0,NaN,प्रतिनिधी,नागपूरपानठेले,बंद,असतानाही,सध्या,मोठ्या,प्रमाणात,खर्रा,उपलब्ध,होत,आहे,NaN,NaN,NaN
1,NaN,NaN,ED,NEO,O,O,O,O,O,O,O,O,O,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NaN,बंदी,असताना,गुटखाही,नागपुरात,मोठ्या,प्रमाणात,येत,असल्याचेही,पुढे,आले,NaN,NaN,NaN,NaN
4,NaN,NaN,O,O,O,NEL,O,O,O,O,O,O,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,1,अन्न,व,औषध,प्रशासन,विभागाने,केलेल्या,तपासणीत,७१,नमुने,असुरक्षित,असल्याचे,आढळले,NaN,NaN
7,NaN,NaN,O,O,O,BNEO,INEO,O,O,NEM,O,O,O,O,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3,NaN,सहआयुक्त,चंद्रकांत,पवार,यांच्या,मार्गदर्शनाखाली,या,कारवाया,करण्यात,आल्या,NaN,NaN,NaN,NaN,NaN


In [ ]:
data=data_1

In [ ]:
data=data.fillna('replace')

In [ ]:
sentences=[]
words=[]
tag_words=[]
tags=[]
check=[]
columns=[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,'Unnamed: 15']
for i in range(0, len(data)):

      # if(i%3==0):
      if(data['Unnamed: 0'][i]!='replace'):
        if data['CHECK'][i]==1:
          check.append(1)
        else:
          check.append(0)
        for j in range(0, len(columns)):
          words.append(data[columns[j]][i])
          tag_words.append(data[columns[j]][i+1])
        sentences.append(words)
        tags.append(tag_words)
        words=[]
        tag_words=[]
        # data=data.drop(i+1)

In [ ]:
for i in range(0,len(sentences)):
  sentences[i].append('END')
  tags[i].append('END')

In [ ]:
len(sentences)

25002

In [ ]:
sentences

In [ ]:
tags

In [ ]:
len(tags)

25002

In [ ]:
len(check)

25002

In [ ]:
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
from fast_ml.model_development import train_valid_test_split

In [ ]:
df = pd.DataFrame(list(zip(sentences, tags)),
               columns =['Sentence', 'Tag'])

In [ ]:
df

,Sentence,Tag
0,"[प्रतिनिधी, नागपूरपानठेले, बंद, असतानाही, सध्य...","[ED, NEO, O, O, O, O, O, O, O, O, O, replace, ..."
1,"[बंदी, असताना, गुटखाही, नागपुरात, मोठ्या, प्रम...","[O, O, O, NEL, O, O, O, O, O, O, replace, repl..."
2,"[अन्न, व, औषध, प्रशासन, विभागाने, केलेल्या, तप...","[O, O, O, BNEO, INEO, O, O, NEM, O, O, O, O, r..."
3,"[सहआयुक्त, चंद्रकांत, पवार, यांच्या, मार्गदर्श...","[ED, NEP, NEP, O, O, O, O, O, O, replace, repl..."
4,"[सुगंधित, तंबाखू, घातकसुगंधित, तंबाखू, मोठ्या,...","[O, O, O, O, O, O, O, O, O, O, O, replace, rep..."
...,...,...
24997,"[पण, वरिष्ठ, स्तरावर, अजून, कटुता, दिसत, आहे ,...","[O, O, O, O, O, O, O, replace, replace, replac..."
24998,"[बंडखोरी, आणि, वादानंतर, परतलेले, सचिन, पायलट,...","[O, O, O, O, BNEP, INEP, O, ED, BNEP, INEP, O,..."
24999,"[यानंतर, अशोक, गहलोत, यांनी, बोलकं, वक्तव्यं, ...","[O, NEP, NEP, O, O, O, O, replace, replace, re..."
25000,"[१९, आमदारांशिवायही, आम्ही, बहुमत, सिद्ध, केले...","[NEM, O, O, O, O, O, O, O, BNEP, INEP, O, repl..."


In [ ]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = "Tag",
                                                                            train_size=0.86, valid_size=0.06, test_size=0.08, random_state=3)

In [ ]:
train_x=X_train.values.tolist()
test_x=X_test.values.tolist()
valid_x=X_valid.values.tolist()

In [ ]:
train_y=y_train.tolist()
test_y=y_test.tolist()
valid_y=y_valid.tolist()

In [ ]:
final_train_x=[]
for i in range(0, len(train_x)):
  final_train_x.append(train_x[i][0])

final_test_x=[]
for i in range(0, len(test_x)):
  final_test_x.append(test_x[i][0])

final_valid_x=[]
for i in range(0, len(valid_x)):
  final_valid_x.append(valid_x[i][0])

In [ ]:
def convert_to_list(df_x, df_y):
  list_sentences=[]
  list_tags=[]
  for i in range(0, len(df_x)):
    for j in range(0, len(df_x[i])):
      list_sentences.append(df_x[i][j])
      list_tags.append(df_y[i][j])
  return list_sentences, list_tags

In [ ]:
X_train_frame, y_train_frame=convert_to_list(final_train_x, train_y)
X_test_frame, y_test_frame=convert_to_list(final_test_x,test_y)
X_valid_frame, y_valid_frame=convert_to_list(final_valid_x, valid_y)

In [ ]:
df_train = pd.DataFrame(list(zip(X_train_frame, y_train_frame)),
               columns =['words', 'labels'])

df_test = pd.DataFrame(list(zip(X_test_frame, y_test_frame)),
               columns =['words', 'labels'])

df_valid = pd.DataFrame(list(zip(X_valid_frame, y_valid_frame)),
               columns =['words', 'labels'])

In [ ]:
def preprocess(df):
  for i in range(0, len(df)):
    if df['labels'][i]=='NED':
      df['labels'][i]='BNED'
    if df['labels'][i]=='NEP' :
      df['labels'][i]='BNEP'
    if df['labels'][i]=='ED':
      df['labels'][i]='BED'
    if df['labels'][i]=='NEL':
      df['labels'][i]='BNEL'
    if df['labels'][i]=='NEO':
      df['labels'][i]='BNEO'
    if df['labels'][i]=='NEM':
      df['labels'][i]='BNEM'
    if df['labels'][i]=='NETI':
      df['labels'][i]='BNETI'
    if df['labels'][i]=='BOE' or df['labels'][i]=='IOE' or df['labels'][i]=='OE' or df['labels'][i]=='OO' or df['labels'][i]=='o':
      df['labels'][i]='O'


  df.drop(df[df['labels'] == ':'].index, inplace = True)
  df.drop(df[df['labels'] == '<<'].index, inplace = True)
  df.drop(df[df['labels'] == '<'].index, inplace = True)
  df.drop(df[df['labels'] == ''].index, inplace = True)
  df.drop(df[df['labels'] == '-'].index, inplace = True)
  df.drop(df[df['labels'] == ' '].index, inplace = True)
  df.drop(df[df['labels'] == 'replace'].index, inplace = True)

  df=df.reset_index()
  df=df.drop('index',axis=1)
  return df

In [ ]:
df_train=preprocess(df_train)
df_test=preprocess(df_test)
df_valid=preprocess(df_valid)

In [ ]:
def sentenceID(df):
  df['sentence_id']=np.nan
  df['sentence_id'][0]='1'
  s_count=2
  for i in range (0,len(df)):
    if df['words'][i]=='END':
      df['sentence_id'][i+1]=str(s_count)
      s_count=s_count+1
  df=df.fillna(method='ffill')
  for i in range(0, len(df)):
    df['words'][i]=str(df['words'][i])
  df.drop(df[df['labels'] == 'END'].index, inplace = True)
  df=df.reset_index()
  df=df.drop('index',axis=1)
  return df

In [ ]:
df_train=sentenceID(df_train)
df_test=sentenceID(df_test)
df_valid=sentenceID(df_valid)

In [ ]:
df_train.head(30)

,words,labels,sentence_id
0,यादरम्यान,O,1
1,वर्षानुवर्षे,O,1
2,विसर्जनानंतर,O,1
3,स्वच्छता,O,1
4,करणाऱ्यांच्या,O,1
5,मनामध्ये,O,1
6,करोनामुळे,O,1
7,वाढलेल्या,O,1
8,प्लास्टिक,O,1
9,वापराची,O,1


In [ ]:
def remove_space(df):
  words=[]
  words=df_train['words'][0].split()
  for i in range(0,len(df)):
    words=df['words'][i].split()
    df['words'][i]=''.join(words)
  return df

In [ ]:
df_train=remove_space(df_train)
df_test=remove_space(df_test)
df_valid=remove_space(df_valid)

In [ ]:
df_train['labels'].value_counts()

O        171956
BNEM       5824
BNEP       4775
BNEL       4461
BNEO       2741
INEP       2135
BNED       1937
INEO       1435
INEM       1228
BED         838
BNETI       633
INED        529
INEL        488
IED         165
INETI       111
Name: labels, dtype: int64

In [ ]:
df_test['labels'].value_counts()

O        16262
BNEM       523
BNEP       428
BNEL       407
BNEO       256
BNED       191
INEP       183
INEO       129
INEM        97
BED         74
BNETI       63
INED        53
INEL        40
IED         18
INETI       10
Name: labels, dtype: int64

In [ ]:
df_valid['labels'].value_counts()

O        12119
BNEM       404
BNEP       322
BNEL       293
BNEO       178
BNED       141
INEP       135
INEO        90
INEM        84
BED         61
BNETI       43
INED        41
INEL        36
IED         14
INETI        5
Name: labels, dtype: int64

In [ ]:
# # writing to Excel
# datatoexcel = pd.ExcelWriter('train.csv')

# write DataFrame to excel
df_train.to_csv("train_iob.txt", index = False,sep='\t')

In [ ]:
df_test.to_csv("test_iob.txt", index = False,sep='\t')

In [ ]:
df_valid.to_csv("valid_iob.txt", index = False,sep='\t')

In [ ]:
training_data_bert=df_train
testing_data_bert=df_test
tuning_data_bert=df_valid

## Transformers

In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
model_args = NERArgs()
# model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.train_batch_size = 16

In [ ]:
new_tags=['O','BNETI','BNEM','BNEP','BNEL','BNED','BNEO','BED', 'INETI', 'INEM', 'INEP', 'INEL', 'INED', 'INEO', 'IED']

In [ ]:
new_tags

['O',
 'BNETI',
 'BNEM',
 'BNEP',
 'BNEL',
 'BNED',
 'BNEO',
 'BED',
 'INETI',
 'INEM',
 'INEP',
 'INEL',
 'INED',
 'INEO',
 'IED']

In [ ]:
from sklearn.metrics import accuracy_score

## 1) Multicase Bert(mbert)

In [ ]:
custom_labels = new_tags

mbert = NERModel(
    "bert", "bert-base-multilingual-cased", labels=custom_labels, args =model_args
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [ ]:
for i in range(0,2):
  !rm -rf /content/outputs
  mbert.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = mbert.eval_model(testing_data_bert)
  print(result)

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)

0.9703426124197002

## 2) Indic Bert

In [ ]:
custom_labels = new_tags

indicbert = NERModel(
    "albert", "ai4bharat/indic-bert", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForTokenClassification: ['predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and a

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

In [ ]:
for i in range(0,10):
  !rm -rf /content/outputs
  indicbert.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = indicbert.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

{'eval_loss': 0.29906198906549736, 'precision': 0.6141253507951356, 'recall': 0.5615911035072711, 'f1_score': 0.5866845397676497}


In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

[904, 1484]
2


In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)

0.9279770623617041

## 3) Roberta-marathi

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

roberta_marathi = NERModel(
     "roberta", "flax-community/roberta-base-mr", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at flax-community/roberta-base-mr were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at flax-community/roberta-base-mr and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task 

Downloading:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

In [ ]:
for i in range(0, 10):
  !rm -rf /content/outputs
  roberta_marathi.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = roberta_marathi.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)

## 4) Xlm-roberta

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

xlmroberta = NERModel(
     "xlmroberta", "xlm-roberta-base", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

In [ ]:
for i in range(2):
  !rm -rf /content/outputs
  xlmroberta.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = xlmroberta.eval_model(testing_data_bert)
  print(result)

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)

## 5) Custom model 1 Maha Bert

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

mr_custom_1 = NERModel(
     "bert", "/content/drive/MyDrive/CUSTOM_MODELS(MARATHI)/ckpt_mbert_cased/ckpt_mbert_cased", labels=custom_labels, args =model_args
)

Some weights of the model checkpoint at /content/drive/MyDrive/CUSTOM_MODELS(MARATHI)/ckpt_mbert_cased/ckpt_mbert_cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from t

In [ ]:
for i in range(0,9):
  !rm -rf /content/outputs
  mr_custom_1.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = mr_custom_1.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1344 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/188 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.1477634613297123, 'precision': 0.8356095093576126, 'recall': 0.8502316006176016, 'f1_score': 0.8428571428571427}


In [ ]:
import tensorflow as tf
checkpoint = tf.train.Checkpoint(model=mr_custom_1, optimizer=)
mr_custom_1.save('/content/drive/MyDrive/mahaBERT_IOB')

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)

## 6) Custom model 4 Maha Roberta

In [ ]:
custom_labels = new_tags

mr_custom_5 = NERModel(
     "xlmroberta", "/content/drive/MyDrive/Marathi_Custom_Models/CUSTOM_MODELS(MARATHI)/ckpt_roberta/ckpt_roberta", labels=custom_labels, args=model_args

)

Some weights of the model checkpoint at /content/drive/MyDrive/Marathi_Custom_Models/CUSTOM_MODELS(MARATHI)/ckpt_roberta/ckpt_roberta were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Marathi_Custom_Models/CUSTOM_MODELS(MARATHI)/ckpt_roberta/ck

In [ ]:
for i in range(15):
  !rm -rf /content/outputs
  mr_custom_5.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = mr_custom_5.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

{'eval_loss': 0.4528250584048313, 'precision': 0.5996292863762743, 'recall': 0.553464499572284, 'f1_score': 0.5756227758007118}


In [ ]:
!zip -r /content/maharoberta_data.zip /content/outputs/best_model

  adding: content/outputs/best_model/ (stored 0%)
  adding: content/outputs/best_model/scheduler.pt (deflated 50%)
  adding: content/outputs/best_model/special_tokens_map.json (deflated 50%)
  adding: content/outputs/best_model/model_args.json (deflated 60%)
  adding: content/outputs/best_model/config.json (deflated 53%)
  adding: content/outputs/best_model/eval_results.txt (deflated 21%)
  adding: content/outputs/best_model/sentencepiece.bpe.model (deflated 49%)
  adding: content/outputs/best_model/training_args.bin (deflated 48%)
  adding: content/outputs/best_model/pytorch_model.bin (deflated 7%)
  adding: content/outputs/best_model/optimizer.pt (deflated 70%)
  adding: content/outputs/best_model/tokenizer_config.json (deflated 55%)


In [ ]:
for i in range(15):
  !rm -rf /content/outputs
  mr_custom_5.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = mr_custom_5.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

{'eval_loss': 0.45000619772923756, 'precision': 0.6347569955817378, 'recall': 0.5530367835757057, 'f1_score': 0.5910857142857142}


In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

[904, 1478, 1484, 1489]
4


In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)

0.9299671721119275

## 7) Custom model 5 Maha-Albert

In [ ]:
custom_labels = new_tags

mr_albert_2epoch = NERModel(
     "albert", "/content/drive/MyDrive/Marathi_Custom_Models/CUSTOM_MODELS(MARATHI)/marathi_albert_finaldatatwoepoch/marathi_albert_finaldatatwoepoch", labels=custom_labels, args=model_args

)

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Marathi_Custom_Models/CUSTOM_MODELS(MARATHI)/marathi_albert_finaldatatwoepoch/marathi_albert_finaldatatwoepoch and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for i in range(10):
  !rm -rf /content/outputs
  mr_albert_2epoch.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = mr_albert_2epoch.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

{'eval_loss': 0.24023585099348566, 'precision': 0.560035445281347, 'recall': 0.5406330196749358, 'f1_score': 0.5501632208922743}


In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

[]
0


In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)

0.9220214813435152

## 15) Custom Model-8(l3cube-pune/marathi-albert-v2)

In [ ]:
custom_labels = new_tags

ravirajoshi_test_full_albert_scratch = NERModel(
    "albert", "l3cube-pune/marathi-albert-v2", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/838 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/126M [00:00<?, ?B/s]

Some weights of the model checkpoint at l3cube-pune/marathi-albert-v2 were not used when initializing AlbertForTokenClassification: ['predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at l3cube-pune/marathi-albert-v2 and are newly initialized: ['classifier.bias', 'classifie

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/531 [00:00<?, ?B/s]

In [ ]:
for i in range(0,10):
  !rm -rf /content/outputs
  ravirajoshi_test_full_albert_scratch.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = ravirajoshi_test_full_albert_scratch.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1343 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/188 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.08900050783343613, 'precision': 0.8457844183564568, 'recall': 0.8623503808487486, 'f1_score': 0.8539870689655171}


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1343 [00:00<?, ?it/s]

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)